# Class Project 3 - Integrating large datasets (Claims and UMLS Data)

Description: In this program we pull 2009 exclusive patient data from a synthetic SynPUF data file containing information on patient insurance claims, including Patient ID, total costs of outpatient care, predominate disagnosis (ICD9 code), and number of physicains visited. We then futher use the ICD9 code to extract assosiated diagnosis and treatment medication information from a set of UMLS files, MRCONSO.RFF and MRREF.RRF, and then create and write said data to a csv file.

Logic: In this program, we use pandas to take in a SynPUF insurance claims file and associated MRCONSO and MRREL UMLS files and create respective smaller sized dataframes by stripping only the required data for further analysis. We next collect all unique PatientIDs associated and create a series of dictionaries mapping the IDs to needed 2009 data in the insurance claim file, where we calculate the total amount of claim payments, unique doctor visits, and most frequent billed diagnosis by ICD9 code. Moreover, we take the most frequently billed ICD9 code and collect the associated ConceptID in the MRCONSO file representing the diagnosis. Furthermore, we use these curated ConceptIDs to search in the MRREL file for the medications used to treat these diagnoses. Finally, we take said final Concept IDs and return the associated medications in the MRCONSO file and write all required data to a cvs file, where each row represents a unique PatientID. 

In [1]:
import csv
import pandas as pd
import re
import numpy as np
import xml.etree.ElementTree as ET
import os

#takes in outpatient data file, reindexes columns, and strips out needed rows and columns to reduce size
def getAndStripDataFrame():
    df = ''
    indexes = [str(x) for x in range(0, 22)]
    with open("/opt/class/medicare/outpatient.csv") as file:
    
        #strips out needed columns
        df = pd.read_csv(file, delimiter=",",usecols=[0,1,2,3,4,5,6,7,8,9,10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],low_memory=False)
    
        df.columns = indexes
        rows_to_drop = []
    
    #strips out 2009 data
    for (index, row) in df.iterrows():
        start_date = row.loc['3']
        end_date = row.loc['4']
        year = '2009'

        #uses regular expressions to match dates
        start = re.match(r"\d{4}", str(start_date))
        end = re.match(r"\d{4}", str(end_date))

        if(start != None and end != None):
            start_value = start.string[0:4]
            end_value = end.string[0:4]

            if(start_value != year or end_value != year):
                rows_to_drop.append(index)
        else:
            rows_to_drop.append(index)

    df = df.drop(rows_to_drop)

    return df

#gets unique Patient IDs from row index 0 in outpatient dataframe
def getPatientIds(dataFrame):
    return list(dataFrame["0"].unique())

#gets total 2009 claim payment amounts for each uique Patient ID and returns dictionary of IDs and total values
def getPaymentAmount(dataFrame):
    patient_ids=  dataFrame["0"].values
    amounts = pd.Series(dataFrame["6"].values, index=patient_ids)
    
    amount_dic = {}
    
    #iterates through amounts series by each unique PatientID totals claim amounts
    for index, value in amounts.iteritems():
        if(index not in amount_dic.keys()):
            amount_dic[index] = value #add new value
        else:
            amount_dic[index] += value #add up additional values
            
    return amount_dic

#gets total unique number of doctors per patient
def getDoctors(dataFrame):
    doc_dict = {}
    used_doctors = []
    ID_STACK = []
    
    #gets all doctor visits and returns a list of respective doc lists
    for item in dataFrame.iterrows():
        row = item[1]
        ID = row["0"]
        doc1 = row["8"]
        doc2 = row["9"]
        doc3 = row["10"]
        doctors = [doc1, doc2, doc3]
        
        #creates abd uses a stack to check for later PatientIDs to make sure later doctor visits 
        #not initially clustered with IDs) in the file aren't missed
        if ID not in doc_dict.keys():
            used_doctors = []
            doc_dict[ID] = 0
            ID_STACK = []
            ID_STACK.append(ID)

        elif ID_STACK[-1] != ID:
            used_doctors = []
            ID_STACK = []
            ID_STACK.append(ID)

        #avoids missing values and adds a count for each unique doctor per unique PatienID
        for doc in doctors:
            if(doc == doc and doc not in used_doctors):
                doc_dict[ID] += 1
                used_doctors.append(doc)

    return doc_dict

#gets max number of unique diagnosis ICD9 codes
def getMax(dataFrame):
    num_dict = {}
    patients = {}
    
    #iterates through outpatient file to grab PatientIDs and diagnoses
    for item in dataFrame.iterrows():
        row = item[1] #creates series object
        ID = row["0"]
        diagnosis = row.tolist()[12:21]
        
        #check if ID in patient keys then creates new dictionary
        if(ID not in patients.keys()):
            num_dict = {}
            patients[ID] = num_dict
        
        #iterates through diagnosis list and counts frequency of diagnoses
        for num in diagnosis:
            if num == num and  num not in num_dict.keys():
                num_dict[num] = 1
            
            #otherwise append count   
            elif num == num:
                num_dict[num] += 1

    return pickHighestDiagnosis(patients)

#picks highest count ICD9
def pickHighestDiagnosis(patients):
    num_list = []
    
    #For each Patient ID
    for patient in sorted(patients.keys()):
            nums = list(patients[patient].values()) #gest all ICD9
            currentPatient = patients[patient]
            
            #chooses diagnosis with count over 1 (likely max values)
            if(max(nums) > 1):
                max_nums = []
                #
                for num_value in currentPatient.keys():
                    
                    #if ISCD9 equals the max of list then append to max_nums list
                    if(currentPatient[num_value] == max(nums)):
                        max_nums.append(num_value)
                
                #pass to addNumbers function
                num_list = addNumbers(max_nums, num_list)

            else:
                num_list = addNumbers(list(currentPatient.keys()), num_list)
    
    
    patient_ids = list(sorted(patients.keys()))
    counter = 0
    patients_numbers = {}
    
    #iterates through PatientIDs and returns max diagnosis
    for ID in patient_ids:
        patients_numbers[ID] = num_list[counter]
        counter += 1

    return patients_numbers

#returns most frequent diagnosis
def addNumbers(values, num_list):
    counter = 0
    for num in values:
        
        #if diagnosis doesn't contain "V" or "E", then return number value only; otherwise return last tied max value
        if(num.startswith("V") == False and num.startswith("E") == False):
            num_list.append(num)
            break;

        if(counter == len(values) -1):
            num_list.append(num)
            break;

        counter += 1

    return num_list

#gets ConceptID in MRCONSO file matching most frequent diagnosis
def getCI(num_dict):
    df = pd.read_csv("/opt/class/umls/icd/MRCONSO.RRF", header = None, delimiter="|", encoding="latin-1",low_memory=False)

    #creates list of needed column values
    concepts = df[0].values
    numbers = df[10].values
    con_nums = list(zip(concepts, numbers))
    concept_numbers = dict((x, y) for x, y in con_nums)

    #adds decimal after third character of diagnosis code and then returns associated concept value in MRCONSO file
    concept_dict = {}
    for ID in num_dict.keys():
        num = num_dict[ID]
        if(len(num) > 3):
            num = num[0:3] + "." + num[3:]

        for concept, compare_num in concept_numbers.items():
            if(num == compare_num):
                concept_dict[ID] = concept
                break;

    return concept_dict

#gets ConceptIDs and diagnosis from MRREL file (strips only needed data)
def filter(data_frame):
    concept_lefts = []
    concept_rights = []
    
    #creates series object and iterates through each row to get lists of ConceptIDs and diagnoses
    for item in data_frame.iterrows():
        row = item[1]
        concept_left = row[0]
        concept_right = row[1]
        diagnosis = row[2]
        
        #only gets data where may_be_treated_by is i the diagnosis column
        if(diagnosis == "may_be_treated_by"):
            concept_lefts.append(concept_left)
            concept_rights.append(concept_right)

    return list(zip(concept_lefts, concept_rights))


def getConcepts(patient_concepts, rel_concepts):

    conso_concepts = {}
    
    #iterates through the right stripped ConceptID columns obtained from the MRREL file and takes left ConceptID
    for ID, concept in patient_concepts.items():
        for concept_left, concept_right in rel_concepts:
            if concept == concept_right:
                conso_concepts[ID] = concept_left
                break;
                
    return  conso_concepts

#gets medication based on associated ConceptID and returs dictionary of said ConceptID per each unique PatientID
def getMedication(conso_concepts):
    
    df = pd.read_csv("/opt/class/umls/icd/MRCONSO.RRF", header = None, delimiter="|", encoding="latin-1",low_memory=False)
    

    medications = df[14].values 
    concepts = df[0].values 

    con_med = list(zip(concepts, medications)) # [(c000534858, "Treated by blah..."), (c0033495, "") ]
    medication_dic = {}
    
    #iterates through MRCONSO file by collected MRREL left column assosiated ConcetIDs and grabs associated medication
    for ID, concept in conso_concepts.items():
        for con, med in con_med:
            if concept == con:
                medication_dic[ID] = med
                break;

    return medication_dic

df = getAndStripDataFrame()
rel = ''

with open("/opt/class/umls/icd/MRREL.RRF") as MRREL:
    rel = pd.read_csv(MRREL, delimiter="|", usecols=[0, 4, 7])
    
concepts = filter(rel)
IDS = getPatientIds(df)
amount = getPaymentAmount(df)
doctors = getDoctors(df)
number_dict = getMax(df)
concept_dict = getCI(number_dict)

conso_concepts = getConcepts(concept_dict, concepts)

medication_dic = getMedication(conso_concepts)

mega_dic = {}

#creates mega-list and associated mega-dictionary of all needed data by PatientID
for ID in IDS:
    currentPatientData = []
    currentPatientData.append(ID)
    currentPatientData.append(amount[ID])
    currentPatientData.append(number_dict[ID])
    
    #returns "nan" if no medication is linked to diagnosis in MRCONSO file and returns linked medication otherwise
    if(ID not in medication_dic.keys()):
        currentPatientData.append("nan")
    else:
        currentPatientData.append(medication_dic[ID])

    currentPatientData.append(doctors[ID])

    mega_dic[ID] = currentPatientData

    
csv.file = open("Jake77_ClassProject3.csv", "w+")
writer = csv.writer(csv.file)
header = ["PatientID", "Total cost of outpatient care", "Predominant diagnosis", "Treatment Medication", "Number of Physicians visted"]
writer.writerow(header)

#writes all values from mega-dicionary (all needed data) to csv file by rows on unique PatienIDs
for patientData in mega_dic.values():
    writer.writerow(patientData)

csv.file.close()

Output: Outputs a csv file in the home directory containing the specified 2009 SynPUF and UMLS data needed from the outpatient dataset and corresponding UMLS files using the Patient ID Identifiers and associated Concept IDs. The code is robust in that it returns a complete result for all unique 2009 Patient ID's listed in the given SynPUY outpatient file. The runtime of the program in Jupyter is 10 minutes, 45 seconds.